In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


root = "/z5s/cv/scan/ame.backup2"
dataDir = "/z5s/cv/cvdata/NewYorkFly/Test"

cfgname1 = "nyf.dss.cuda@3.batch@16.lr@40.loss@infoNCE" # 78.85
cfgname2 = "nyf.dssentropy.cuda@1.batch@16.lr@40.loss@infoNCEWeightedDirect.gridn@2.ne" # 92.8

df1 = pd.read_csv(os.path.join(root, cfgname1, "eval/detail.csv"))
df2 = pd.read_csv(os.path.join(root, cfgname2, "eval/detail.csv"))

qposDf = pd.read_csv(os.path.join(dataDir, "query.csv"))
rposDf = pd.read_csv(os.path.join(dataDir, "reference.csv"))

f1 = df1.loc[df1["mFiles"] != df1["gtFiles"]]
t2 = df2.loc[df2["mFiles"] == df1["gtFiles"]]

improvement = t2.loc[t2["qFiles"].isin(f1["qFiles"])]
cdf = improvement.merge(f1[["qFiles", "mFiles"]].rename(columns={"mFiles": "s4g"}), on="qFiles", how="left")

cdf = cdf.merge(qposDf[["name", "lat", "long"]].rename(columns={"name": "qFiles", "lat": "qx", "long": "qy"}), on="qFiles", how="left")
cdf = cdf.merge(rposDf[["name", "lat", "long"]].rename(columns={"name": "gtFiles", "lat": "gx", "long": "gy"}), on="gtFiles", how="left")
cdf = cdf.merge(rposDf[["name", "lat", "long"]].rename(columns={"name": "s4g", "lat": "mx", "long": "my"}), on="s4g", how="left")

cdf.to_csv("/z5s/cv/cvdata/NewYorkFly/Test/tmp.txt", index=False)

def calcDist(parray1, parray2):
    parray1 = np.radians(parray1)
    parray2 = np.radians(parray2)
    geoMat = haversine_distances(parray1, parray2) * 6387209.7
    return np.diagonal(geoMat)

cdf.loc[:, "gtDistance"] = calcDist(cdf[["qx", "qy"]], cdf[["gx", "gy"]])
cdf.loc[:, "s4gDistance"] = calcDist(cdf[["qx", "qy"]], cdf[["mx", "my"]])
cdf = cdf.loc[(cdf["gtDistance"] - cdf["s4gDistance"]).abs() > 0.1]

outputDir = "/z5s/cv/cvdata/NewYorkFly/Test/tmp"

for i in range(cdf.shape[0]):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    entry = cdf.iloc[i]
    qFile = os.path.join(dataDir, "query_images", entry["qFiles"])
    gtFile = os.path.join(dataDir, "reference_images", entry["gtFiles"])
    mFile = os.path.join(dataDir, "reference_images", entry["s4g"])
    qpos = "({},{})".format(entry["qx"], entry["qy"])
    gpos = "({},{})".format(entry["gx"], entry["gy"])
    mpos = "({},{})".format(entry["mx"], entry["my"])
    gdist = entry["gtDistance"]
    mdist = entry["s4gDistance"]

    for ax, imgFile, pos, dist, tag in zip(axes, [qFile, gtFile, mFile], [qpos, gpos, mpos], [None, gdist, mdist], ["Query", "Ours(GroundTruth)", "Sample4Geo(False)"]):
        ax.imshow(Image.open(imgFile))
        ax.xaxis.set_major_locator(ticker.NullLocator())
        ax.yaxis.set_major_locator(ticker.NullLocator())
        title = "{}-{}".format(tag, pos) if dist is None else "{}, distance={:.3f}\n{}".format(tag, dist, pos)
        ax.set_title(title)

    fig.savefig(os.path.join(outputDir, "{}.png".format(i)))
